In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
fraudcar = pd.read_csv("data/training_set_fd.csv",sep=";")
fraudcar

In [ ]:
# ANALISI

In [ ]:
print(f'Number of istances: {fraudcar.shape[0]}')

In [ ]:
print(f'Is there any missing values: {fraudcar.isna().any().any()}')

In [ ]:
print(fraudcar["fraud"].value_counts())
sns.countplot(fraudcar, x = "fraud")

In [ ]:
fraudcar = fraudcar[fraudcar["age_of_driver"] < 91]
fraudcar["age_of_driver"].plot.hist(bins=100)

In [ ]:
ages = [18,21,25,30,40,50,90]
categories = pd.cut(fraudcar['age_of_driver'], ages)
categories

pd.pivot_table(fraudcar[fraudcar["fraud"] == 1], 'fraud', index=[categories], aggfunc='count').plot.bar()

In [ ]:
"""3. Considerare il dataset originale e considerare la divisione in uomini e donne, e all’interno di
ogni gruppo la divisione in under o over 40 (si includano anche le persone con quaranta anni in
questo gruppo). Indicare a quale gruppo occorre fare maggiore attenzione perché è più facile
avere una frode all’interno di esso (motivare la decisione) """

In [ ]:
fraudcar = pd.read_csv("data/training_set_fd.csv",sep=";")
fraudcar

In [ ]:
ages = [0,39,100]
categories = pd.cut(fraudcar['age_of_driver'], ages)
categories

pd.pivot_table(fraudcar[fraudcar["fraud"] == 1], 'fraud', index=['gender',categories], aggfunc='count').plot.bar()

# Pay attention on the group of female over 40 because it have the max percentage of fraud

In [ ]:
"""4. Verificare con un opportuno diagramma se è vero che la distribuzione delle frodi aumenta
all’aumentare del pagamento richiesto per l’indennizzo (attributo claim_est_payout) (punti 2)"""
new_dataset = fraudcar.copy()
new_dataset["claim_est_payout"].hist(bins=100)

new_datasetnf = new_dataset[new_dataset["fraud"] == 0]
new_datasetnf["claim_est_payout"].hist(bins=100)

new_datasetf = new_dataset[new_dataset["fraud"] == 1]
new_datasetf["claim_est_payout"].hist(bins=100)

In [ ]:
# Trasformazione e Predizione

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

In [ ]:
"""1. Si vuole predire il valore di fraud sulla base degli attributi presenti nel dataset. Ricaricare il
dataset originale, eliminare eventuali attributi inutili (giustificare la scelta), eliminare le istanze
che contengono valori nulli, rendere tutti gli attributi numerici, e dividerlo in modo che 2/3
degli elementi siano contenuti in un nuovo dataset “train” e 1/3 nel dataset “test”.
Allenare il train con il modello Decision Tree e valutare l’accuracy ottenuta calcolata sia sul
dataset train sia sul dataset test. Confrontare i risultati ottenuti con quelli ottenuti con una
predizione basata sul modello Logistic Regression (ignorare eventuali warning). Effettuare alcune
considerazioni sui risultati ottenuti, tenendo in considerazione anche l’analisi della confusion
matrix e la predizione effettuata da un dummy classifier. L’accuratezza è la metrica migliore
per misurare la qualità del modello in questo scenario, o sarebbe opportuno utilizzare un’altra
metrica? (punti 4)"""

In [ ]:
fraudcar = pd.read_csv("data/training_set_fd.csv",sep=";")
fraudcar

In [ ]:
fraudcar.drop(["claim_number"], inplace=True, axis=1) # "claim_number" è semplicemente un numero seriale
fraudcar = fraudcar.dropna()

In [ ]:
y = fraudcar["fraud"]
X = fraudcar.drop(columns=["fraud"]) # remove the target
X

In [ ]:
to_encode = ["gender", "living_status",'claim_day_of_week','accident_site','channel','vehicle_category','vehicle_color']
enc = preprocessing.OrdinalEncoder()

for col in to_encode:
    X[col] = enc.fit_transform(pd.DataFrame(X[col]))

X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [ ]:
# Decision tree
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

result = dt.predict(X_test)
print("Accuracy with decision tree:")
print(f'accuracy on train: {accuracy_score(y_train, dt.predict(X_train))}')
print(f'accuracy on test: {accuracy_score(y_test,result)}')

confusionmatrix = confusion_matrix(y_test,result)
sns.heatmap(confusionmatrix,annot=True,fmt="d")

In [ ]:
# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)

result = lr.predict(X_test)
print("Accuracy with logistic regression:")
print(f'accuracy on train: {accuracy_score(y_train, lr.predict(X_train))}')
print(f'accuracy on test: {accuracy_score(y_test,result )}')

confusionmatrix = confusion_matrix(y_test,result)
sns.heatmap(confusionmatrix,annot=True,fmt="d")

In [ ]:
# DummyClassifier
dclf = DummyClassifier()
dclf.fit(X_train, y_train)

result = dclf.predict(X_test)
print("Accuracy with Dummy Classifier:")
print(f'accuracy on train: {accuracy_score(y_train, dclf.predict(X_train))}')
print(f'accuracy on test: {accuracy_score(y_test,result )}')

confusionmatrix = confusion_matrix(y_test,result)
sns.heatmap(confusionmatrix,annot=True,fmt="d")

In [ ]:
"""L’accuratezza è la metrica migliore
per misurare la qualità del modello in questo scenario, o sarebbe opportuno utilizzare un’altra
metrica? """
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

result = dt.predict(X_test)
print("Precision, recall and F1-score with decision tree:")
print(f'F1-score on test: {f1_score(y_test,result)}')
print(f'Precision on test: {precision_score(y_test,result)}')
print(f'Recall on test: {recall_score(y_test,result)}')


In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

result = lr.predict(X_test)
print("Precision, recall and F1-score with decision tree:")
print(f'F1-score on test: {f1_score(y_test,result)}')
print(f'Precision on test: {precision_score(y_test,result)}')
print(f'Recall on test: {recall_score(y_test,result)}')

In [ ]:
"""2. Confrontare l’accuratezza ottenuta nel punto precedente con l’accuratezza si ottiene con un
una 10 Fold cross validation. (punti 1)"""

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
dt = DecisionTreeClassifier()
lr = LogisticRegression()

print(f'Average accuracy on Decision Tree with 10 Fold cross validation: {cross_val_score(dt,X, y, cv=10).mean()}')
print(f'Average accuracy on Logistic Regression with 10 Fold cross validation: {cross_val_score(lr,X, y, cv=10).mean()}')

In [ ]:
"""3. Trovare i parametri migliori del classificatore Logistic Regression. Agire sui parametri penalty
e C. Verificare se l’accuratezza che si ottiene con la nuova configurazione supera quella
standard ottenuta al punto 1 (punti 4)"""

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
logr = LogisticRegression()
parameters = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.1, 0.5, 1., 2., 3.],
}

clf = GridSearchCV(logr, parameters, cv=10, n_jobs=-1)
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [ ]:
logr = LogisticRegression(C=clf.best_params_["C"], penalty=clf.best_params_["penalty"])
lr.fit(X_train, y_train)

print("Accuracy with logistic regression:")
print(f'accuracy on train: {accuracy_score(y_train, lr.predict(X_train))}')
print(f'accuracy on test: {accuracy_score(y_test,lr.predict(X_test) )}')

In [ ]:
""" 4.Introdurre una discretizzazione degli attributi claim_est_payout e vehicle_price, e
utilizzare la funzione MaxAbsScaler per scalare i valori del dataset tra 0 e 1 e confrontare se
l’accuratezza ottenuta con il Decision Tree Classifier e con la Logistic Regression migliora"""


In [ ]:
fraudcar_copy = X.copy()

In [ ]:
kb = preprocessing.KBinsDiscretizer(n_bins=30, encode="ordinal", strategy="uniform")
fraudcar_copy["claim_est_payout"] = kb.fit_transform(fraudcar_copy["claim_est_payout"].values.reshape(-1, 1))
kb = preprocessing.KBinsDiscretizer(n_bins=20, encode="ordinal", strategy="uniform")
fraudcar_copy["vehicle_price"] = kb.fit_transform(fraudcar_copy["vehicle_price"].values.reshape(-1, 1))
fraudcar_copy

In [ ]:
max_abs_scaler = preprocessing.MaxAbsScaler()

X_scaled = pd.DataFrame(max_abs_scaler.fit_transform(fraudcar_copy),columns=fraudcar_copy.columns)
X_scaled

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=1/3)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

result = lr.predict(X_test)
print("Accuracy with logistic regression:")
print(f'accuracy on train: {accuracy_score(y_train, lr.predict(X_train))}')
print(f'accuracy on test: {accuracy_score(y_test,result )}')


In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

result = dt.predict(X_test)
print("Accuracy with decision tree:")
print(f'accuracy on train: {accuracy_score(y_train, dt.predict(X_train))}')
print(f'accuracy on test: {accuracy_score(y_test,result)}')


In [ ]:
"""5. Creare una pipeline in cui si aggiungano al dataset normalizzato due colonne che
rappresentano i valori degli attributi claim_est_payout e vehicle_price discretizzati in 10
intervalli. Si valuti se l’accuratezza migliora utilizzando LogisticRegression come modello (punti
2)"""

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import FeatureUnion, FunctionTransformer

In [ ]:
def identity(X):
  return X


Identity = FunctionTransformer(identity, validate=True)

discretizer = ColumnTransformer([("payout", preprocessing.KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"), ["claim_est_payout"]),
                                 ("vehicle_price", preprocessing.KBinsDiscretizer(n_bins=10, encode="ordinal", strategy="uniform"), ["vehicle_price"])])
union = FeatureUnion([("discretizer", discretizer),
                      ("allelse", Identity)])

In [ ]:
model_pipeline = LogisticRegression()

In [ ]:
my_pipeline = Pipeline(steps=[('discretizer', union),
                                ('model', model_pipeline)
                             ], verbose = True)
my_pipeline

In [ ]:
my_pipeline.fit(X_train, y_train)

preds = my_pipeline.predict(X_test)
 
accuracy_score(y_test, preds)

In [ ]:
"""6. Aggiungere alla pipeline la funzione SelectKBest (https://scikit-
learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html?highlight=selec
tkbest#sklearn.feature_selection.SelectKBest). Utilizzare la funzione di gridSearchCV per
selezionare il K migliore e anche gli intervalli migliori in cui discretizzare i valori di
claim_est_payout e vehicle_price (punti 3).
"""

my_pipeline = Pipeline(steps=[('discretizer', union),
                              ('selectkbest',SelectKBest()),
                                ('model', model_pipeline)
                             ], verbose = True)

my_pipeline

In [ ]:
parameters = {
    'discretizer__discretizer__payout__n_bins': [10, 20, 30, 40, 50],
    'discretizer__discretizer__vehicle_price__n_bins': [5, 10, 15, 20, 25, 30],
    'selectkbest__k': [1, 5, 10, "all"]
}

clf = GridSearchCV(my_pipeline, parameters,cv = 5, n_jobs=-1)
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
my_pipeline.set_params(**clf.best_params_)

In [ ]:
my_pipeline.fit(X_train, y_train)

preds = my_pipeline.predict(X_test)
 
accuracy_score(y_test, preds)

In [ ]:
"""7. Creare una nuova pipeline che applichi un simpleImputer (anziché la rimozione delle righe), al
dataset iniziale. Si aggiunga questa pipeline a quella del punto 6 e si valuti la strategia migliore
tra mean, median e most frequent (si decida una configurazione qualsiasi per gli altri
parametri.(punti 3)."""

In [196]:
from sklearn.impute import SimpleImputer

In [198]:
fraudcar = pd.read_csv("data/training_set_fd.csv",sep=";")
fraudcar

,claim_number,age_of_driver,gender,marital_status,annual_income,high_education_ind,address_change_ind,living_status,claim_day_of_week,accident_site,past_num_of_claims,channel,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud
0,1,46,M,1.0,38301,1,1,Rent,Friday,Local,1,Broker,7530.940993,9.0,Compact,12885.452350,white,16161.33381,0
1,3,21,F,0.0,30445,0,1,Rent,Thursday,Highway,1,Online,2966.024895,4.0,Large,29429.452180,white,28691.96422,0
2,4,49,F,0.0,38923,0,1,Own,Tuesday,Local,0,Broker,6283.888333,3.0,Compact,21701.181950,white,22090.94758,1
3,5,58,F,1.0,40605,1,0,Own,Thursday,Local,3,Broker,6169.747994,4.0,Medium,13198.273440,other,38329.58106,1
4,6,38,M,1.0,36380,1,0,Rent,Tuesday,Highway,0,Broker,4541.387150,7.0,Medium,38060.211220,gray,25876.56319,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17993,29993,69,M,1.0,42338,1,0,Own,Saturday,Local,0,Broker,4806.841803,6.0,Medium,32144.571850,black,16868.53410,0
17994,29996,35,F,0.0,35579,1,1,Own,Monday,Local,1,Phone,4089.794471,3.0,Medium,9468.510601,blue,31500.99689,1
17995,29997,27,F,1.0,32953,0,1,Rent,Saturday,Local,2,Online,2225.803056,5.0,Medium,64974.369590,black,44536.25506,0
17996,29999,52,F,1.0,39519,1,0,Own,Friday,Highway,1,Broker,9560.744844,3.0,Compact,24983.268110,white,50093.75959,0


In [199]:
fraudcar.drop(["claim_number"], inplace=True, axis=1) 
y = fraudcar["fraud"]
X = fraudcar.drop(columns=["fraud"]) # remove the target

to_encode = ["gender", "living_status",'claim_day_of_week','accident_site','channel','vehicle_category','vehicle_color']
enc = preprocessing.OrdinalEncoder()

for col in to_encode:
    X[col] = enc.fit_transform(pd.DataFrame(X[col]))

X

,age_of_driver,gender,marital_status,annual_income,high_education_ind,address_change_ind,living_status,claim_day_of_week,accident_site,past_num_of_claims,channel,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight
0,46,1.0,1.0,38301,1,1,1.0,0.0,1.0,1,0.0,7530.940993,9.0,0.0,12885.452350,6.0,16161.33381
1,21,0.0,0.0,30445,0,1,1.0,4.0,0.0,1,1.0,2966.024895,4.0,1.0,29429.452180,6.0,28691.96422
2,49,0.0,0.0,38923,0,1,0.0,5.0,1.0,0,0.0,6283.888333,3.0,0.0,21701.181950,6.0,22090.94758
3,58,0.0,1.0,40605,1,0,0.0,4.0,1.0,3,0.0,6169.747994,4.0,2.0,13198.273440,3.0,38329.58106
4,38,1.0,1.0,36380,1,0,1.0,5.0,0.0,0,0.0,4541.387150,7.0,2.0,38060.211220,2.0,25876.56319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17993,69,1.0,1.0,42338,1,0,0.0,2.0,1.0,0,0.0,4806.841803,6.0,2.0,32144.571850,0.0,16868.53410
17994,35,0.0,0.0,35579,1,1,0.0,1.0,1.0,1,2.0,4089.794471,3.0,2.0,9468.510601,1.0,31500.99689
17995,27,0.0,1.0,32953,0,1,1.0,2.0,1.0,2,1.0,2225.803056,5.0,2.0,64974.369590,0.0,44536.25506
17996,52,0.0,1.0,39519,1,0,0.0,0.0,0.0,1,0.0,9560.744844,3.0,0.0,24983.268110,6.0,50093.75959


In [210]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

,age_of_driver,gender,marital_status,annual_income,high_education_ind,address_change_ind,living_status,claim_day_of_week,accident_site,past_num_of_claims,channel,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight
12267,31,1.0,1.0,34343,1,1,0.0,6.0,1.0,1,0.0,4051.654962,1.0,1.0,33535.139840,2.0,7155.056956
4724,48,0.0,1.0,38697,1,0,1.0,3.0,0.0,4,2.0,3993.668101,3.0,2.0,6501.850356,2.0,15457.687950
11850,25,1.0,1.0,32181,1,1,1.0,4.0,1.0,0,1.0,3459.085062,8.0,1.0,29892.625420,4.0,22479.895940
8176,23,0.0,0.0,31348,1,1,1.0,1.0,2.0,0,2.0,4667.899259,5.0,1.0,41970.481610,4.0,17030.171690
12939,53,1.0,1.0,39700,1,1,0.0,1.0,1.0,4,0.0,3452.156550,5.0,2.0,40619.675270,0.0,4729.131641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,33,1.0,1.0,34971,1,0,1.0,4.0,2.0,0,0.0,2398.934033,3.0,2.0,11090.996720,0.0,25922.991170
12723,61,0.0,1.0,41112,1,0,0.0,1.0,1.0,0,0.0,6631.229388,5.0,2.0,16229.408560,6.0,77989.164350
2942,47,1.0,0.0,38503,1,1,0.0,3.0,0.0,1,2.0,3835.439122,2.0,1.0,7595.449425,2.0,7551.419140
814,33,1.0,1.0,34944,1,1,0.0,0.0,1.0,0,2.0,8580.858862,8.0,2.0,28101.481870,3.0,21018.634680


In [219]:
parameters = {
    'old_pipe__discretizer__discretizer__payout__n_bins': [20],
    'old_pipe__discretizer__discretizer__vehicle_price__n_bins': [10],
    'old_pipe__selectkbest__k': ['all'],
    'impute__strategy': ["mean", "median", "most_frequent"]
}

discretizer = ColumnTransformer([("payout", preprocessing.KBinsDiscretizer(n_bins=30, encode="ordinal", strategy="uniform"), [X_train.columns.get_loc("claim_est_payout")]),
                                 ("vehicle_price", preprocessing.KBinsDiscretizer(n_bins=20, encode="ordinal", strategy="uniform"), [X_train.columns.get_loc("vehicle_price")])])

union = FeatureUnion([("discretizer", discretizer),
                      ("allelse", Identity)])
pipeline = Pipeline(steps=[("discretizer", union),
                           ("estimator", LogisticRegression())])

extended_pipeline = Pipeline(steps=[("discretizer", union),
                                    ("selectkbest", SelectKBest()),
                                    ("estimator", LogisticRegression()),
                                   ])
impute_pipeline = Pipeline(steps=[("impute", SimpleImputer()),
                                   ("old_pipe", extended_pipeline)])

gd = GridSearchCV(impute_pipeline, parameters)
gd.fit(X_train, y_train)

/usr/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimiz

GridSearchCV(estimator=Pipeline(steps=[('impute', SimpleImputer()),
                                       ('old_pipe',
                                        Pipeline(steps=[('discretizer',
                                                         FeatureUnion(transformer_list=[('discretizer',
                                                                                         ColumnTransformer(transformers=[('payout',
                                                                                                                          KBinsDiscretizer(encode='ordinal',
                                                                                                                                           n_bins=30,
                                                                                                                                           strategy='uniform'),
                                                                                                                          [11]),
                                                                                                                         ('vehicle_price',
                                                                                                                          KBinsDiscretizer(encode='ordinal',
                                                                                                                                           n_bins=20,
                                                                                                                                           strategy='uniform'),
                                                                                                                          [1...
                                                                                         FunctionTransformer(func=<function identity at 0x705957012660>,
                                                                                                             validate=True))])),
                                                        ('selectkbest',
                                                         SelectKBest()),
                                                        ('estimator',
                                                         LogisticRegression())]))]),
             param_grid={'impute__strategy': ['mean', 'median',
                                              'most_frequent'],
                         'old_pipe__discretizer__discretizer__payout__n_bins': [20],
                         'old_pipe__discretizer__discretizer__vehicle_price__n_bins': [10],
                         'old_pipe__selectkbest__k': ['all']})

In [220]:
gd.best_params_

{'impute__strategy': 'most_frequent',
 'old_pipe__discretizer__discretizer__payout__n_bins': 20,
 'old_pipe__discretizer__discretizer__vehicle_price__n_bins': 10,
 'old_pipe__selectkbest__k': 'all'}

In [222]:
preds = gd.predict(X_test)
 
accuracy_score(y_test, preds)

0.8406666666666667